In [60]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import cosine_distances
import numpy as np

In [17]:

import gensim, pickle
from gensim.models.keyedvectors import KeyedVectors
LdaModel = gensim.models.ldamodel.LdaModel


In [10]:
ruta_word_embedding = '../data/wiki.multi.en.vec'
##wordembedding = gensim.models.Word2Vec.load(ruta_word_embedding)
wordembedding = KeyedVectors.load_word2vec_format(ruta_word_embedding, binary=False)

In [15]:
##Load Gensim Model

lda_model_collecion_1 = LdaModel.load("../data/cambridge_analytica/collection_I/collection_1_gensim.model")
lda_model_collecion_2 = LdaModel.load("../data/cambridge_analytica/collection_II/collection_2_gensim.model")

In [20]:
with open('../data/cambridge_analytica/collection_I/collection_1_sent_topics_sorteddf_mallet_ldamodel', 'rb') as f:
    most_relevant_documents_collection_1 = pickle.load(f)
most_relevant_documents_collection_1 = most_relevant_documents_collection_1[['Topic_Num','Topic_Perc_Contrib','text']]


with open('../data/cambridge_analytica/collection_II/collection_2_sent_topics_sorteddf_mallet_ldamodel', 'rb') as f:
    most_relevant_documents_collection_2 = pickle.load(f)
most_relevant_documents_collection_2 = most_relevant_documents_collection_2[['Topic_Num','Topic_Perc_Contrib','text']]


In [48]:
'''My own tokenizer '''
import unidecode
from string import punctuation
from string import digits
punctuation+="¡¿<>'`"
punctuation+='"'

#Remove digits and puntuaction
remove_digits = str.maketrans(digits, ' '*len(digits))#remove_digits = str.maketrans('', '', digits)
remove_punctuation = str.maketrans(punctuation, ' '*len(punctuation))#remove_punctuation = str.maketrans('', '', punctuation)
remove_hashtags_caracter = str.maketrans('#', ' '*len('#'))
#las palabras de los hashtag se mantiene, pero no el simbolo. 


def text_cleaner(tweet):
    tweet = tweet.translate(remove_digits)
    #tweet = tweet.lower() it wasn't a good idea,, we lost a lot of
    tweet = tweet.translate(remove_punctuation)
    tweet = tweet.translate(remove_hashtags_caracter)
    tweet = tweet.lower()
    tweet = unidecode.unidecode(tweet)
    #tweet = tweet.strip().split()
    #filtered_words = [word for word in tweet if word not in stopWords]
    #corpus[id_tweet]= filtered_words
    #id_tweet+=1
    tweet = tweet.split()
    return tweet


In [65]:
def get_dicts_relevant_keywords_documents(lda_model,df_relevant_documents, n_terms):
    num_topics = lda_model.num_topics
    #create dictionary of top keywords 
    topKeywordsDict = {}
    for topic_id in range(num_topics):
        topKeywordsDict[topic_id] = []
        for term, probability in lda_model.show_topic(topic_id,topn=n_terms):
            topKeywordsDict[topic_id].append({
                "term":term,
                "probability":probability
            })

    #create dictionary of relevant documents
    relevantDocumentsDict = {}
    for index,row in df_relevant_documents.iterrows():
        topic_id = int(row['Topic_Num'])
        if topic_id not in relevantDocumentsDict:
            relevantDocumentsDict[topic_id]=[]
        relevantDocumentsDict[topic_id].append({
            'topic_perc_contrib':row['Topic_Perc_Contrib'],
            'text':row['text']
        })
    return (topKeywordsDict, relevantDocumentsDict)

def getDocumentVector(text, wordembedding):
    #preprocesar    
    #encontrar palabras en word embedding
    #ponderas palabras TF-IDF
    document_vector = 0.0
    words_found = 0.0
    for word in text_cleaner(text):
        if word in wordembedding:
            document_vector+=wordembedding.wv[word] #aqui hay que ponderar
            words_found+=1
    return document_vector/words_found

        
    
def get_topkeywords_relevantdocuments_vectors(wordembedding, lda_model,most_relevant_documents,  n_terms): #n_terms : numero de top keywords a considerar
    topKeywordsDict, relevantDocumentsDict = get_dicts_relevant_keywords_documents(lda_model, most_relevant_documents, n_terms)

    ### Create top keyword vector per topic
    #create keyword vector
    topkeywords_vectors_dict = {}
    num_topics = lda_model.num_topics
    for topic_id in range(num_topics):
        topkeywords_vector = 0
        ranking = 1.0
        for item in topKeywordsDict[topic_id]:
            if item['term'] in wordembedding: #no todas las palabras aparecerán en el ranking, que hacer con el resto
                #print(item['term'], item['probability'])
                topkeywords_vector += wordembedding.wv[item['term']]/ranking
            else:
                print(item['term']," position:",ranking)
            ranking+=1
        topkeywords_vectors_dict[topic_id] = topkeywords_vector
        
    relevantdocuments_vectors_dict = {}
    for topic_id in range(num_topics):
        relevantDocumentsvector = 0.0
        for item in relevantDocumentsDict[topic_id]:
            #quizas esto hacerlo para los primero 100 docs, 500 docs, el resto es un % pequeño que se pierde, optimizar
            #revisar si multiplicar por la contribucion es lo correcto, eso no da 1 o si? OHHHHHH, habria que sacar todooos los docs no solo los 100 primeros
            relevantDocumentsvector+= float(item['topic_perc_contrib'])*getDocumentVector(item['text'], wordembedding)
        relevantdocuments_vectors_dict[topic_id] = relevantDocumentsvector
        
    return (topkeywords_vectors_dict, relevantdocuments_vectors_dict)


In [66]:
def get_topic_vectors(wordembedding, lda_model,most_relevant_documents,  n_terms, lambda_):
    num_topics = lda_model.num_topics
    topkeywords_vectors_dict, relevantdocuments_vectors_dict = get_topkeywords_relevantdocuments_vectors(wordembedding, lda_model,most_relevant_documents,  n_terms)
    final_topic_vectors_dict = dict()
    for topic_id in range(num_topics):
        final_topic_vector = lambda_*topkeywords_vectors_dict[topic_id]+(1-lambda_)*relevantdocuments_vectors_dict[topic_id]
        final_topic_vectors_dict[topic_id] = final_topic_vector
    return final_topic_vectors_dict

In [67]:
def get_matrix(wordembedding, lda_model_1,most_relevant_documents_1,lda_model_2,most_relevant_documents_2, n_terms, lambda_):
    
    final_topic_vectors_dict_1 =  get_topic_vectors(wordembedding, lda_model_1,most_relevant_documents_1,  n_terms, lambda_)
    final_topic_vectors_dict_2 =  get_topic_vectors(wordembedding, lda_model_2,most_relevant_documents_2,  n_terms, lambda_)
    
    topic_similarity_matrix = []
    for i in range(lda_model_1.num_topics):
        row = []
        for j in range(lda_model_2.num_topics):
            topic_i = final_topic_vectors_dict_1[i].reshape(1,-1)
            topic_j = final_topic_vectors_dict_2[j].reshape(1,-1)
            row.append(float(cosine_similarity(topic_i,topic_j)))
        topic_similarity_matrix.append(row)
    topic_similarity_matrix= np.asarray(topic_similarity_matrix)
    return topic_similarity_matrix

In [89]:
topn = 30
lambda_ = 0.8


i = 0.0
matrices_dict = dict()
while i <=1.01:
    lambda_ = round(i,2)
    print(lambda_)
    matrix = get_matrix(wordembedding, lda_model_collecion_1, most_relevant_documents_collection_1, lda_model_collecion_2, most_relevant_documents_collection_2,topn, lambda_)
    matrices_dict[lambda_] = matrix
    i+=0.01
 

0.0


C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0
retweete  position: 6.0
retweete  position: 15.0
retweeted  position: 26.0
gdpr  position: 30.0
bigdata  position: 4.0
deletefacebook  position: 13.0
bigdata  position: 1.0
machinelearne  position: 3.0
datascience  position: 6.0
deeplearne  position: 10.0
deeplearning  position: 26.0
dataanalytic  position: 27.0
topinfluence  position: 28.0
machinelearning  position: 29.0
0.01


C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0
retweete  position: 6.0
retweete  position: 15.0
retweeted  position: 26.0
gdpr  position: 30.0
bigdata  position: 4.0
deletefacebook  position: 13.0
bigdata  position: 1.0
machinelearne  position: 3.0
datascience  position: 6.0
deeplearne  position: 10.0
deeplearning  position: 26.0
dataanalytic  position: 27.0
topinfluence  position: 28.0
machinelearning  position: 29.0
0.02


...

deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0
retweete  position: 6.0
retweete  position: 15.0
retweeted  position: 26.0
gdpr  position: 30.0
bigdata  position: 4.0
deletefacebook  position: 13.0
bigdata  position: 1.0
machinelearne  position: 3.0
datascience  position: 6.0
deeplearne  position: 10.0
deeplearning  position: 26.0
dataanalytic  position: 27.0
topinfluence  position: 28.0
machinelearning  position: 29.0
0.99


C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0
retweete  position: 6.0
retweete  position: 15.0
retweeted  position: 26.0
gdpr  position: 30.0
bigdata  position: 4.0
deletefacebook  position: 13.0
bigdata  position: 1.0
machinelearne  position: 3.0
datascience  position: 6.0
deeplearne  position: 10.0
deeplearning  position: 26.0
dataanalytic  position: 27.0
topinfluence  position: 28.0
machinelearning  position: 29.0
1.0


C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:52: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
C:\Users\gonza\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:33: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


deletefacebook  position: 21.0
retweete  position: 14.0
bigdata  position: 1.0
machinelearne  position: 9.0
datascience  position: 13.0
deletefacebook  position: 9.0
twitt  position: 29.0
retweete  position: 6.0
retweete  position: 15.0
retweeted  position: 26.0
gdpr  position: 30.0
bigdata  position: 4.0
deletefacebook  position: 13.0
bigdata  position: 1.0
machinelearne  position: 3.0
datascience  position: 6.0
deeplearne  position: 10.0
deeplearning  position: 26.0
dataanalytic  position: 27.0
topinfluence  position: 28.0
machinelearning  position: 29.0


In [90]:
matrices_dict

{0.0: array([[0.96770757, 0.85364223, 0.97768849, 0.91522002, 0.90502191,
         0.98005593, 0.96814501, 0.95625162, 0.89008987, 0.93868363,
         0.78171235, 0.90576875, 0.94801211, 0.97304964, 0.95291841],
        [0.95439708, 0.87655133, 0.97175592, 0.89949334, 0.90001827,
         0.9547174 , 0.9712069 , 0.97398156, 0.87955832, 0.98068702,
         0.7756871 , 0.92108965, 0.95364857, 0.97006142, 0.93500322],
        [0.94308603, 0.85379863, 0.92739528, 0.95039421, 0.92225206,
         0.97001481, 0.9571054 , 0.95050687, 0.93781006, 0.90900803,
         0.85293329, 0.9256773 , 0.96131849, 0.97651839, 0.98081619],
        [0.93969125, 0.84940886, 0.91258675, 0.95977831, 0.90480864,
         0.95023185, 0.94694161, 0.94119668, 0.91711146, 0.92422676,
         0.81146538, 0.89158463, 0.95418489, 0.9600417 , 0.96551895],
        [0.93772531, 0.85834265, 0.94152564, 0.93974984, 0.91136825,
         0.9560045 , 0.95749319, 0.95733285, 0.92527926, 0.95898664,
         0.80344814, 0.90

In [91]:
with open('../data/cambridge_analytica/matrix_collection_1_2', 'wb') as f:
            pickle.dump(matrices_dict, f)